# **CH02.2. Seq2Seq**

#### **`Paper Info`** : Sequence to Sequence Learning with Neural Networks
#### $ \hspace{1.75cm} - $ Ilya Sutskever, Oriol Vinyals, Quoc V. Le et al. (NeurIPS/2014)

<b></b>

> ## **요약(Summary)**

| Item | Description |
|------|------|
| Research topic | 기계번역을 위한 variable-length 시퀀스 간 매핑을 Encoder–Decoder LSTM으로 end-to-end 학습 |
| Core idea | source를 고정 차원 context vector로 encoding하고 decoder를 conditional Language Model로 학습하여 target을 순차 생성하며 <br> source reversing으로 minimal time lag를 감소 |
| Key findings | $ \cdot{} $ source reversing이 최적화를 크게 용이하게 하여 BLEU 향상 관찰 <br> $ \cdot{} $ 작은 beam size에서도 효과적이며 ensemble이 phrase-based SMT를 상회 <br> $ \cdot{} $ PCA 시각화에서 context vector가 syntax와 semantics를 동시에 반영 |
| Contributions | $ \cdot{} $ alignment나 복잡한 feature engineering 없이 end-to-end NMT 제시 <br> $ \cdot{} $ 4-layer LSTM, ≈3.8e8 params(수백 M급), 8-GPU 병렬 등 대규모 학습 recipe 제시 <br> $ \cdot{} $ 길이 증가에 대한 성능 저하가 완만하고 표현 공간 구조를 실증 |

<b></b>

> ## **연구 배경(Motivation & Prior Work)**

#### **(1) 기존 연구 및 한계점** :
##### $ \hspace{0.15cm} \cdot{} $ DNN 기반 번역은 position-specific parameters로 word order 변화를 유연하게 처리하지 못함
##### $ \hspace{0.15cm} \cdot{} $ 고정 길이 입력·출력 제약으로 길이가 다른 문장 간 mapping이 어려움
##### $ \hspace{0.15cm} \cdot{} $ Phrase-based SMT는 alignment와 복잡한 feature engineering에 의존하며 신경망의 표현학습 이점을 직접 활용하기 어려움
#### **(2) 연구 목표** :
##### $ \hspace{0.15cm} \cdot{} $ 임의 길이 source를 임의 길이 target으로 직접 매핑하는 end-to-end 학습 프레임 구축
##### $ \hspace{0.15cm} \cdot{} $ word order와 문맥 의존성을 효과적으로 포착하여 SMT 대비 번역 품질을 향상
##### $ \hspace{0.15cm} \cdot{} $ 대규모 코퍼스에서 안정적으로 학습 가능한 실용적 학습 전략을 제시
#### **(3) 제안된 방법론** :
##### $ \hspace{0.15cm} \cdot{} $ Encoder–Decoder LSTM으로 source를 context vector로 요약하고 decoder를 conditional Language Model로 학습하여 target을 순차 생성하도록 설계
##### $ \hspace{0.15cm} \cdot{} $ <EOS> 토큰으로 문장 종료를 모델링하여 전체 시퀀스에 대한 확률분포를 정의
##### $ \hspace{0.15cm} \cdot{} $ source를 역순으로 입력하는 preprocessing으로 minimal time lag를 축소하여 최적화 난이도를 완화

<b></b>

> ## **방법론(Method)**

#### **(1) 문제정의(Conditional sequence modeling)** :
##### $ \hspace{0.15cm} \cdot{} $ 입력 $ x_{1:T} $ 과 출력 $ y_{1:T'} $ 에 대해 조건부 확률을 체인룰로 전개
##### $ \hspace{0.15cm} \Rightarrow{} \displaystyle{} P(y_{1:T'} \mid x_{1:T}; \theta{}) = \prod_{t=1}^{T'} P\big(y_{t} \mid y_{1:t-1}, \, c(x_{1:T}); \theta{}\big) \;\; \text{ where } \, c(x_{1:T}) \in \mathbb{R}^{d} \text{ is context vector}, \, \theta{} \text{ are parameters } $
##### $ \hspace{0.15cm} \cdot{} $ 디코더 단계별 분포는 softmax로 계산
##### $ \hspace{0.15cm} \Rightarrow{} \displaystyle{} P(y_{t}=w \mid \cdot) = \frac{\exp{}\big(\mathbf{u}_{w}^{\top} \mathbf{h}_{t} + b_{w}\big)}{\sum_{w' \in \mathcal{V}_{y}} \exp{}\big(\mathbf{u}_{w'}^{\top} \mathbf{h}_{t} + b_{w'}\big)} \;\; \text{ where } \, \mathbf{h}_{t} \text{ is decoder hidden state}, \, \mathbf{u}_{w}, b_{w} \text{ are output parameters} $
#### **(2) Encoder–Decoder LSTM 구조** :
##### $ \hspace{0.15cm} \cdot{} $ Encoder는 $ x_{1}, \ldots, x_{T} $ 를 순차 처리하고 최종 상태를 $ c = c(x_{1:T}) $ 로 사용
##### $ \hspace{0.15cm} \cdot{} $ Decoder는 초기 상태를 $ c $ 로 정하고 시작 토큰부터 <EOS> 가 나올 때까지 $ y_{t} $ 를 순차 생성
#### **(3) <EOS> 기반 시퀀스 종료 및 정규화** :
##### $ \hspace{0.15cm} \cdot{} $ 디코더가 <EOS> 를 생성하면 시퀀스 종료로 간주하여 임의 길이 문장 공간에서의 확률 정규화가 가능
##### $ \hspace{0.15cm} \Rightarrow{} \displaystyle{} \sum_{T'=1}^{\infty} \sum_{y_{1:T'} \in \mathcal{V}_{y}^{T'}} P(y_{1:T'} \mid x_{1:T}) = 1 \;\; \text{ where } \, \text{the model emits <EOS> with nonzero probability at each step} $
#### **(4) 학습목적함수(Negative log-likelihood)** :
##### $ \hspace{0.15cm} \cdot{} $ 데이터셋 $ \mathcal{D} = \{(S^{(n)}, T^{(n)})\}_{n=1}^{N} $ 에 대해 조건부 로그우도를 최대화
##### $ \hspace{0.15cm} \Rightarrow{} \displaystyle{} \mathcal{J}(\theta{}) = - \sum_{n=1}^{N} \sum_{t=1}^{|T^{(n)}|} \log{} P\big(y^{(n)}_{t} \mid y^{(n)}_{1:t-1}, c(S^{(n)}); \theta{}\big) \;\; \text{ where } \, S^{(n)} \text{: source}, \, T^{(n)} \text{: target} $
#### **(5) Decoding(Beam search)** :
##### $ \hspace{0.15cm} \cdot{} $ 최대확률 시퀀스 근사화를 위해 누적 로그확률 기준의 beam search 사용
##### $ \hspace{0.15cm} \Rightarrow{} \displaystyle{} \hat{T} = \arg\max_{T} \sum_{t=1}^{|T|} \log{} P\big(y_{t} \mid y_{1:t-1}, c(x_{1:T}); \theta{}\big) \;\; \text{ where } \, \text{beam size } B \text{ controls hypothesis pruning} $
#### **(6) Source reversing(minimal time lag 감소)** :
##### $ \hspace{0.15cm} \cdot{} $ 입력을 역순으로 제공하여 초기 디코딩 단계와 관련된 encoder 단어의 시간 간격을 단축
##### $ \hspace{0.15cm} \cdot{} $ 그래디언트 전달 경로가 짧아져 optimization이 쉬워지고 학습이 안정화

<b></b>

> ## **실험(Experiments)**

#### **(1) 실험 설정** :
##### $ \hspace{0.15cm} \cdot{} $ 데이터셋
| Item | Description |
|------|--------------|
| Training dataset | WMT14 En→Fr (cleaned subset) |
| Data(Sentence;row) size | 12M |
| Vocabulary(token) | En 160k, Fr 80k (top-freq), OOV→<UNK> |
| Evaluation dataset | WMT14 test |
| Evaluation size | ? |
##### $ \hspace{0.15cm} \cdot{} $ 모델 및 훈련
| Item | Description |
|------|--------------|
| Model type | 2-LSTM (Encoder/Decoder) |
| Hidden layer | 4-layer LSTM per stack, 1000 cells |
| Embedding dimension | 1000 |
| Optimizer | SGD (no momentum) |
| Learning rate | 0.7 then half decay after 5ep and every 0.5ep |
| Batch size | 128 |
| Regularization | Gradient clipping (norm ≤ 5) |
| Training hardware | 8 GPUs (pipeline + softmax parallelization) |
| Training time | ~10 days |
| Baseline models | Phrase-based SMT (1000-best for reranking) |
| Evaluation metric | BLEU (cased) |
#### **(2) 분석 및 결과** :
##### $ \hspace{0.15cm} \cdot{} $ 단일 모델 direct decoding은 SMT와 유사하거나 약간 낮지만 ensemble로 SMT를 상회
##### $ \hspace{0.15cm} \cdot{} $ source reversing은 단일 모델에서도 큰 폭의 BLEU 향상 제공
##### $ \hspace{0.15cm} \cdot{} $ 작은 beam size(예: 2)에서도 성능 이득 대부분을 획득하는 효율성 관찰
##### $ \hspace{0.15cm} \cdot{} $ 긴 문장에서도 성능 저하가 완만하며 구조적 일관성이 유지됨
##### **(`PLUS`) OOV 및 고정 vocabulary 이슈** :
##### $ \hspace{0.15cm} \cdot{} $ 희귀 단어는 <UNK>로 대체되어 번역 품질에 불리
##### $ \hspace{0.15cm} \cdot{} $ 후속 연구의 subword segmentation이 이를 완화
#### **(3) 결과 해석** :
##### $ \hspace{0.15cm} \cdot{} $ Encoder가 학습한 context vector는 PCA에서 의미적 근접과 구문적 변형에 대한 구조를 보여 표현학습의 효과를 시각화
##### $ \hspace{0.15cm} \cdot{} $ reversing은 minimal time lag를 줄여 RNN 학습의 credit assignment를 촉진하는 최적화 관점의 트릭으로 해석
##### $ \hspace{0.15cm} \cdot{} $ beam search는 로그확률의 누적합을 최대화하는 탐색으로, 작은 빔에서도 실제적 품질을 확보

<b></b>

> ## **결론(Conclusion)**

#### **(1)** Seq2Seq LSTM은 alignment 없이도 end-to-end로 번역을 수행하며 SMT를 실질적으로 대체할 수 있는 성능을 보임
#### **(2)** Source reversing은 간단하지만 강력한 optimization trick으로 일반 Seq2Seq 학습 안정성에 기여
#### **(3)** Context vector는 syntax와 semantics를 함께 포착하나 고정 bottleneck으로 정보 손실 위험이 존재
#### **(4)** OOV와 rare word 취약성은 subword와 주변부 기법으로 보완되며, bottleneck 문제는 이후 Attention 메커니즘으로 해결되는 연구 계보로 이어짐